In [ ]:
%matplotlib inline
from typing import Dict, Tuple, List, Union, Iterable

import matplotlib.pyplot as plt
#import matplotlib.patches as mpatches
#from matplotlib.ticker import AutoMinorLocator
plt.style.use("lm")
import numpy as np
#import scipy.linalg as sl #expm
#import os
#import time
#import random
#import string
#import copy
#from tabulate import tabulate
import samplebase as sb

import path_utils as putils

def plot(f, xRange, *args, **kw):
    yRange = np.fromiter(map(f, xRange), np.float)
    plt.plot(xRange, yRange, *args, **kw)

In [ ]:
data_dir = putils.data_dir(1)
data_dir